In [1]:
import cv2
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import sys
%matplotlib inline
np.set_printoptions(suppress=True)

## Border Detection

In this notebook you will see an implementation of the Canny's Algorithm for border detection. This algorith has the following steps:

1. Gaussian noise reduction
2. X and Y Derivatives
3. Gradient calculation
4. Phase calculation
5. Non-maximum suppression
6. Double threshold
7. Edge Tracking by Hysteresis

## 1. Gaussian Noise Reduction - Convolution Filter

In [2]:
def convolutional_filter(imagen, kernel):
    alto = imagen.shape[0]
    ancho =  imagen.shape[1]
    
    altoKernel = len(kernel[:,0])
    anchoKernel = len(kernel[0,:])

    #parte entera del centro del kernel
    step = int(len(kernel[0,:])/2)

    newImg = np.zeros((alto, ancho, 1))
    
    for i in range(step, alto - step):
        newPixel = 0
        for j in range(step, ancho - step):
            ventana = imagen[i-step:i+2*step, j-step:j+2*step]
            newPixel = 0
            for m in range(0, altoKernel):
                for n in range(0, anchoKernel):
                    newPixel = newPixel + ventana[m, n] * kernel[m, n]
            newImg[i, j] = int(newPixel)
    
    return newImg

## 2. X and Y Derivatives

In [53]:
def dx(image):
    #im = cv2.imread(image)
    #im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    im_gray = image
    height = im_gray.shape[0]
    width = im_gray.shape[1]
    
    new_x = np.zeros((height, width,1))
    for i in range(0, height-1):
        for j in range(0, width):
            new_x[i,j] = im_gray[i+1,j] - im_gray[i,j]
    
    return(new_x)

In [54]:
def dy(image):
    #im = cv2.imread(image)
    #im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    im_gray = image
    
    height = im_gray.shape[0]
    width = im_gray.shape[1]
    
    new_y = np.zeros((height, width,1))
    for i in range(0, height):
        for j in range(0, width-1):
            new_y[i,j] = im_gray[i,j+1] - im_gray[i,j]
    
    return(new_y)        

## 3. Gradient

In [55]:
def gradient(x, y):
    height = x.shape[0]
    width = x.shape[1]
    
    gr = np.zeros((height, width, 1))

    for i in range(0, height):
        for j in range(0, width):
            gr[i,j] = np.sqrt(x[i,j]**2 + y[i,j]**2)
    
    return(gr)

## 4. Phase

In [56]:
def phase(x, y):
    height = x.shape[0]
    width = x.shape[1]
    
    ph = np.zeros((height, width, 1))

    for i in range(0, height):
        for j in range(0, width):
            ph[i,j] = np.where((math.atan2(y[i,j], x[i,j]) * (180/np.pi))<0,
                               (math.atan2(y[i,j], x[i,j]) * (180/np.pi))+360,
                               (math.atan2(y[i,j], x[i,j]) * (180/np.pi)))
    
    return(ph)

## 5. Non-Maximum Supression

In [57]:
def nms(im_gray, ph):
    
    height = ph.shape[0]
    width = ph.shape[1]

    nms = np.zeros((height, width))

    for i in range(1, height-2):
        for j in range(1, width-2):
            nms[i,j] = np.where(
                (((ph[i,j]>=337.5)&(ph[i,j]<=360)) or ((ph[i,j]<22.5)&(ph[i,j]>=0))) or ((ph[i,j]>=157.5) & (ph[i,j]<202.5)),
                np.where(np.argmax(im_gray[i,j-1:j+2])==im_gray[i,j],im_gray[i,j],0),
                np.where(
                    ((ph[i,j]>=22.5) & (ph[i,j]<67.5)) or ((ph[i,j]>=202.5) & (ph[i,j]<247.5)),
                    np.where(np.max(np.array([im_gray[i+1,j-1], im_gray[i,j], im_gray[i-1,j+1]]))==im_gray[i,j],im_gray[i,j],0),
                    np.where(
                        ((ph[i,j]>=67.5) & (ph[i,j]<112.5)) or ((ph[i,j]>=247.5) & (ph[i,j]<292.5)),
                        np.where(np.max(im_gray[i-1:i+2,j])==im_gray[i,j],im_gray[i,j],0),
                        np.where(np.max(np.array([im_gray[i-1,j-1], im_gray[i,j], im_gray[i+1,j+1]]))==im_gray[i,j],im_gray[i,j],0)
                    )
                )
            )
            
    cv2.imwrite('nms.jpg', nms)
    return(nms)

## 6. Thresholding

In [58]:
def binarization(image, threshold):

    im_gray = image
    
    height = im_gray.shape[0]
    width = im_gray.shape[1]
    
    im_binary = np.where(im_gray<=threshold,0,255)
    
    #original image
    cv2.imwrite("im_gray.jpg", im_gray)
    gray = cv2.imread("im_gray.jpg")
    cv2.imshow("Original Image", gray)
    
    #result image
    cv2.imwrite("im_binary.jpg", im_binary)
    return(im_binary)

## Border Detection Algorithm
--- Putting it all together ---

In [77]:
imagen = cv2.imread('iceland.jpg')
imagen = cv2.cvtColor(imagen,cv2.COLOR_BGR2GRAY)
kernelGauss_33 = np.array([[1/16., 2/16., 1/16.], [2/16., 4/16., 2/16.], [1/16., 2/16., 1/16.]])
conv = convolutional_filter(imagen, kernelGauss_33)
der_x = dx(conv)
der_y = dy(conv)
gr = gradient(der_x, der_y)
ph = phase(der_x, der_y)
non_max = nms(gr, ph)
binary = binarization(non_max, 12)

### Original Image

<img src='iceland.jpg'/>

### Result Image ---> Border Detection with a Gaussian Filter

<img src='nms.jpg'/>

In [64]:
kernelGauss_33 = np.array([[1/16., 2/16., 1/16.], [2/16., 4/16., 2/16.], [1/16., 2/16., 1/16.]])
filtrada = convolutional_filter(cv2.imread('car.jpg'), kernelGauss_33)

420
1024


TypeError: only size-1 arrays can be converted to Python scalars